# Introducción

**Resultados de la encuesta de satisfacción de los usuarios del Hospital (Hospital General Dr. Manuel Gea González) que asisten a los servicios de Consulta Externa. La encuesta hace referencia a temas de trato digno hacia los pacientes y sus familiares. Los datos corresponden al año 2018.**

# Librerías.

In [2]:
from __future__ import division

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import unidecode
%matplotlib inline

# Carga de datos.

In [ ]:
df_cons_ext = pd.read_excel('Datasets/EncuestaTratoDigno/ConsultaExterna2018.xlsx', encoding='ISO-8859-1')

In [ ]:
pd.options.display.max_columns = df_cons_ext.shape[1]
df_cons_ext.head()

# Análisis de Variables.

## Variable *Entrevistado*.

In [ ]:
print len(df_cons_ext['Entrevistado']), len(df_cons_ext['Entrevistado'].unique())

### Ya que el número de *Entrevistado* es único (funciona como un ID), podemos eliminarla y quedarnos con el índice.

In [ ]:
df_cons_ext.drop(labels='Entrevistado', axis=1, inplace=True)

# Cambio de Nombres.

## Creamos nuestros nuevos nombres y los metadatos de éstos.

In [ ]:
def cambio_nombre(df):
    nuevos_nombres = []
    for i in df.columns:
        print i
        nuevos_nombres.append(raw_input())
    return nuevos_nombres

In [ ]:
n_nombs = cambio_nombre(df_cons_ext)

**METADATOS:**

**Turno** = Turno de Atención <br>
**Sexo** = Sexo <br>
**Seguro** = Tipo de Seguridad Social <br>
**Tipo** = El entrevistado es <br>
**Edad** = Edad del entrevistado <br>
**Aten_Resp** = ¿Durante su visita a la unidad lo atendieron con respeto? <br>
**No_Aten_Resp** = Seleccione el personal que no lo atendió con respeto <br>
**Cita_4Sem** = El tiempo de espera para conseguir una cita con el especialista fue de cuatro semanas o más <br>
**Coment** = ¿El médico le dio oportunidad de hacer preguntas o comentar sus preocupaciones respecto al motivo por el que acudió a la consulta? <br>
**Explic_M** = ¿El médico le dio explicaciones fáciles de entender? <br>
**Compr_Medicam** = ¿Entendió Usted cómo deberá tomar los medicamentos? <br>
**Compr_Cuid** = ¿Entendió Usted los cuidados que deberá seguir en su casa? <br>
**Consider_Preocup** = El médico al decidir su tratamiento, ¿tomó en cuenta sus necesidades y preocupaciones? <br>
**No_Cita_Pago** = ¿Dejó de asistir a alguna consulta por no poder pagarla? <br>
**No_Trat_Pago** = ¿En algún momento dejó de realizarse estudios o cumplir con su tratamiento(cirugía, rehabilitación) por no poder pagarlo? <br>
**No_Medicam_Pago** = ¿Dejó de tomar algún medicamento por no poder pagarlo? <br>
**Sufic_T** = ¿Considera que el médico pasó suficiente tiempo con Usted durante la consulta? <br>
**Dolor** = Durante su visita a esta unidad médica, ¿tuvo dolor? <br>
**Dolor_Cur** = ¿Su dolor fue atendido? <br>
**Medicam** = En relación a los medicamentos que le recetaron: <br>
**Satisfacc** = ¿Qué tan contento (satisfecho) está con la calidad de la Atención Médica recibida?

In [ ]:
cambio = zip(df_cons_ext.columns.tolist(), n_nombs)

df_cons_ext.rename(columns=dict(cambio), inplace=True)

In [ ]:
df_cons_ext.head()

# Análisis de Información.

## Tipo de datos.

In [ ]:
df_cons_ext.dtypes

## Número de Missings.

In [ ]:
df_cons_ext.isnull().sum()

## Análisis descriptivo de cada variable.

In [ ]:
df_cons_ext.describe(include='all')

### La variable *Turno* tiene el 9.1% de valores missing, ya que el único valor que tienen es 'a' (Matutino), supondremos que los valores faltantes también lo son.

In [ ]:
df_cons_ext['Turno'].value_counts()

In [ ]:
str(round(df_cons_ext['Turno'].isnull().sum()/len(df_cons_ext['Turno'].notnull())*100, 1)) + '%'

In [ ]:
df_cons_ext.drop('Turno', axis=1, inplace=True)

# Ahora cargamos el diccionario de a encuesta.

In [ ]:
df_dicc = pd.read_csv('Datasets/EncuestaTratoDigno/DiccionarioDatos2018.csv', encoding='ISO-8859-1')

In [ ]:
pd.options.display.max_columns = df_dicc.shape[1]
df_dicc.head()

## Como no podemos trabajar con los caracteres, le cambiamos el nombre a la tabla.

In [ ]:
n_nombs = cambio_nombre(df_dicc)

In [ ]:
cambio = zip(df_dicc.columns.tolist(), n_nombs)

df_dicc.rename(columns=dict(cambio), inplace=True)

### Le quitamos los acentos y cualquier caracter que nos impida trabajar, además pasamos todas las palabras a minúsculas.

In [ ]:
for _ in df_dicc.columns:
    df_dicc[_][df_dicc.notnull()[_]] = df_dicc[_][df_dicc.notnull()[_]].apply(lambda x: unidecode.unidecode(x.lower()))

### Observemos que cada columna de la encuesta le corresponde a una del diccionario.

In [ ]:
for a, b in zip(df_cons_ext, df_dicc.columns.tolist()[1:]):
    print 'C. Externa: {}   Diccionario: {}'.format(a, b)

### Entonces, crearemos un diccionario que contega la letra de la opción y lo que significa. Esto para cada columna.

In [ ]:
cont = 1
for j in df_dicc.columns:
    if j != 'Turno':
        globals()['d' + str(cont)] = {}
        for i in df_dicc[j][df_dicc.notnull()[j]]:
            globals()['d' + str(cont)][i.split(')')[0]] = i.split(')')[1][1:]
        cont += 1

### Vemos que contiene cada diccionario

In [ ]:
for _ in range(1, len(df_dicc.columns)):
    print 'd' + str(_) + ': ' + str(eval('d' + str(_)))

### Ahora cambiamos los valores para cada variable con los diccionarios que creamos

In [ ]:
df_dicc.shape[1]

In [ ]:
for _ in range(df_dicc.shape[1] - 1):
    df_cons_ext[df_cons_ext.columns[_]] = df_cons_ext[df_cons_ext.columns[_]].map(globals()['d' + str(_ + 1)])

In [ ]:
df_cons_ext.head()